In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as f
import torch.distributions as distributions
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
from torch.distributions import Normal, TransformedDistribution
from torch.distributions.transforms import TanhTransform
import numpy as np
import gym
import os
from dm_control import suite
from dm_control.suite.wrappers import pixels
from auxiliares import converter_cinza, training_device
device = training_device()
import random
import wandb


In [ ]:
from torch.nn import Linear as linear
from torch.nn import ReLU as relu

class RSSM(nn.Module):
    def xs(self, x, number_of_layers):
        for i in range(number_of_layers):
            x = linear(x.shape[1], x.shape[1])(x)
            x = relu(x)
        return x
    
    def _core(self):
        # 1. Redimensiona o estado estocástico para forma (batch, stoch * classes)
        stoch = stoch.reshape((stoch.shape[0], -1))
        
        # 3. Define o número de blocos para operações agrupadas
        g = self.blocks
        
        # 4. Funções lambda para reorganizar as dimensões usando einops (equivalente a view/reshape no PyTorch)
        #flat2group = lambda x: einops.rearrange(x, '... (g h) -> ... g h', g=g)
        #roup2flat = lambda x: einops.rearrange(x, '... g h -> ... (g h)', g=g)
        
        # 5. Processa cada entrada (determinístico, estocástico e ação) por camadas lineares
        # PyTorch equivalente: nn.Linear seguido de normalização e ativação
        x0 = nn.Sequential(
          nn.Linear(deter, self.hidden),
          nn.ReLU()
        )

        x1 = nn.Sequential(
          nn.Linear(stoch, self.hidden),
          nn.ReLU()
        )

        x2 = nn.Sequential(
          nn.Linear(action, self.hidden),
          nn.ReLU()
        )
        
        # 6. Concatena os resultados e prepara para operações em blocos
        x = torch.cat([x0, x1, x2], -1)[..., None, :].repeat(g, -2)
        x = group2flat(torch.cat([flat2group(deter), x], -1))
        #x = jnp.concatenate([x0, x1, x2], -1)[..., None, :].repeat(g, -2)
        #x = group2flat(jnp.concatenate([flat2group(deter), x], -1))
        
        # 7. Aplica camadas dinâmicas (similar a camadas densas agrupadas)
        for i in range(self.dynlayers):
            x = self.sub(f'dynhid{i}', nn.BlockLinear, self.deter, g, **self.kw)(x)
            x = nn.act(self.act)(self.sub(f'dynhid{i}norm', nn.Norm, self.norm)(x))
        
        # 8. Gera os gates para atualização do estado (similar a GRU)
        x = self.sub('dyngru', nn.BlockLinear, 3 * self.deter, g, **self.kw)(x)
        gates = jnp.split(flat2group(x), 3, -1)
        reset, cand, update = [group2flat(x) for x in gates]
        
        # 9. Atualização do estado determinístico (equações do GRU modificadas)
        reset = nn.Sigmoid(reset)  # Porta de reset
        cand = nn.Tanh(reset * cand)
        update = nn.Sigmoid(update - 1)
        #reset = jax.nn.sigmoid(reset)  # Porta de reset
        #cand = jnp.tanh(reset * cand)  # Candidato para atualização
        #update = jax.nn.sigmoid(update - 1)  # Porta de update (com bias -1)
        
        # 10. Combinação do estado anterior e novo candidato
        deter = update * cand + (1 - update) * deter
        
        return deter